In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import pickle
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
import scipy.stats as stats
from numpy import linspace
from numpy.core.umath import pi

In [2]:
parent_df_path = 'usage_per_user/'

with open(parent_df_path+'joh_df.pkl', 'rb') as f:
    joh_per_usage = pickle.load(f)
    
with open(parent_df_path+'lon_df.pkl', 'rb') as f:
    lon_per_usage = pickle.load(f)
    
with open(parent_df_path+'nyc_df.pkl', 'rb') as f:
    nyc_per_usage = pickle.load(f)
    
with open(parent_df_path+'ran_df.pkl', 'rb') as f:
    ran_per_usage = pickle.load(f)


all_usage = pd.concat([joh_per_usage,lon_per_usage,nyc_per_usage,ran_per_usage],ignore_index = True)

In [3]:
all_usage.drop(labels = all_usage[all_usage.ethnicity.eq('other')].index,inplace =True)
all_usage.reset_index(inplace = True,drop = True)
all_usage

,#️⃣,*️⃣,0️⃣,1️⃣,2️⃣,3️⃣,4️⃣,5️⃣,6️⃣,7️⃣,...,🧺,🧻,🧼,🧽,🧾,🧿,tweets_contain_emoji,total_tweets,gender,ethnicity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,62.0,111.0,female,black
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,female,black
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,566.0,945.0,female,black
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,251.0,444.0,male,white
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,410.0,887.0,female,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,117.0,3121.0,female,hispanic
18685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,551.0,3072.0,female,black
18686,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1055.0,2482.0,male,asian
18687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,905.0,2752.0,female,white


In [4]:
emoji_prop = all_usage.drop(columns = ['tweets_contain_emoji','total_tweets','gender','ethnicity'])
for index, row in emoji_prop.iterrows():
    s = emoji_prop.loc[index].sum()
    if s!=0:
        emoji_prop.loc[index] = emoji_prop.loc[index]/s
    else:
        emoji_prop.loc[index] = 0

emoji_prop_gender = emoji_prop
emoji_prop_gender['gender'] = all_usage['gender']

gender_bias = emoji_prop_gender.loc[emoji_prop_gender['gender'] == 'male'].mean() - emoji_prop_gender.loc[emoji_prop_gender['gender'] == 'female'].mean()

In [5]:
p_value_by_gender = {}

for index,emoji in enumerate (gender_bias.keys()):
   
    X = emoji_prop_gender[emoji]
#     X = X.round(decimals=2)
    Y = emoji_prop_gender['gender']
    observed = pd.crosstab(Y,X) 

    chi2, p, dof, expected = stats.chi2_contingency(observed)
    p_value_by_gender[emoji] = p 


p_value_by_gender_pd = pd.DataFrame(data=sorted(p_value_by_gender.items(), key=lambda kv: kv[1]),columns=['Emoji','P-value'])

for index,emoji in enumerate(p_value_by_gender_pd['Emoji']):
    if gender_bias[emoji] > 0:
        p_value_by_gender_pd.at[index,'gender'] = 'male'
    elif gender_bias[emoji] < 0:
        p_value_by_gender_pd.at[index,'gender'] = 'female'
    else:
        p_value_by_gender_pd.at[index,'gender'] = 'same'

In [6]:
p_value_by_gender_pd[p_value_by_gender_pd.gender.eq('female')][:10]

,Emoji,P-value,gender
0,💕,1.793519e-11,female
1,😘,9.786708e-06,female
2,❤,8.342638e-05,female
3,💗,1.815383e-04,female
4,💜,1.845759e-04,female
5,✨,1.932681e-04,female
6,😍,2.717428e-04,female
7,🌸,3.269687e-04,female
8,💖,6.926273e-04,female
11,💋,3.617213e-03,female


In [7]:
p_value_by_gender_pd[p_value_by_gender_pd.gender.eq('male')][:10]

,Emoji,P-value,gender
9,⚽,0.001855,male
10,🤷‍♂️,0.002020,male
13,🤦‍♂️,0.005314,male
15,👍,0.007604,male
16,🐐,0.008712,male
18,🔴,0.009241,male
21,🏆,0.019617,male
22,🔵,0.027838,male
28,🤦🏻‍♂️,0.033813,male
30,🚀,0.035098,male


In [8]:
len(p_value_by_gender_pd[(p_value_by_gender_pd['P-value'] <= 0.05) & (p_value_by_gender_pd['gender'] == 'male') ])

10

In [9]:
len(p_value_by_gender_pd[(p_value_by_gender_pd['P-value'] <= 0.05) & (p_value_by_gender_pd['gender'] == 'female') ])

23